## Neural Network without ML Libraries ##
Neural networks are a system which is used in deep learning. I wrote a single-layer neural network without using any Machine Learning Libraries. I only used numpy for math and matplotlib for data visualization. 

### Import Libraries ###

In [12]:
import numpy as np

### Define Data ###
I will use fake data in this example.

In [2]:
# Data
X = [[1,1,0,0,1,1,1,1],[1,1,0,0,0,1,1,1],[1,0,0,0,1,1,1,1],[0,1,1,1,0,0,0,1],[0,0,0,1,0,0,0,0],[0,0,0,1,1,0,0,0]]

# Labels - where players choose to attack mostly
Y = [1,1,1,0,0,0]

# What are the different labels
labels = [0, 1]

### Create A Neural Network ###
A Neural Network can be simply created as three arrays: weights, biases, activations, because every layer is consist of matrix operations: Weight * a + bias.

Note: NOTE: This function can create multilayer neural networks.

In [3]:
def create_neural_net(layer_array, input_dims):
    weights = []
    biases = []
    activations = []
    
    for i in range(len(layer_array)):
        node_num = layer_array[i][0]
        weights_of_layer = []
        biases_of_layer = []
        if i == 0:
            last_layer_node_number = input_dims
        else:
            last_layer_node_number = layer_array[i-1][0]
        
        for n in range(0,node_num):
            weights_of_node = []
            for l in range(0, last_layer_node_number):
                weights_of_node.append(1) 
            weights_of_layer.append(weights_of_node)
            biases_of_layer.append(0)
            
        weights.append(weights_of_layer)
        biases.append(biases_of_layer)
        activations.append(layer_array[i][1])
    return [weights, biases, activations]

### Test of create_neural_net() function ###

In [4]:
layer_array = [[len(labels), 'sigmoid']]
input_dims = 8
neural_net = create_neural_net(layer_array, input_dims)

print(' weights:',neural_net[0],'\n\n biases:',neural_net[1],'\n\n activations:', neural_net[2])

 weights: [[[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]] 

 biases: [[0, 0]] 

 activations: ['sigmoid']


### Activation Functions ###
Activation functions are functions that prevent values from getting too low or too high. There are many activation functions but I will wrote two of them: sigmoid and relu.

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    
def sigmoid_deriv(x):
    return x * (1 - x)

def relu(x):
    if x < 0:
        return 0
    else:
        return x

### Predict Using The Neural Network ###
NOTE: Predict and predict_ratio functions are designed to be used both for single-layer and multilayer neural nets. 

In [6]:
def predict_ratio(data, neural_net):
    weights = neural_net[0]
    biases = neural_net[1]
    activations = neural_net[2]
    
    layer_num = len(weights)
    
    for l in range(0, layer_num):
        data = np.dot(weights[l], data)
        for t in range(len(data)):
            data[t] += biases[l][t]
        if activations[l] == 'sigmoid':
            data = sigmoid(data)
        elif activations[l] == 'relu':
            data = relu(data)
        else:
            # If not identified, do it with sigmoid
            data = sigmoid(data)
            print('activation function', activations[l], 'cannot be found. Sigmoid is used')   
    return data

def predict(data, neural_net):
    data = predict_ratio(data, neural_net)
    
    class_num = len(data)
    
    highest_class = None
    highest_class_probability = -1
    
    for i in range(0, class_num):
        if highest_class == None:
            highest_class = i
            highest_class_probability = data[i]
        elif data[i] > highest_class_probability:
            highest_class = i
            highest_class_probability = data[i]
            
    return highest_class, highest_class_probability

In [7]:
# Of course, this prediction is absulutly wrong because we didn't trained the network yet
predict(X[1], neural_net)

(0, 0.9933071490757153)

### Train Network ###
The most important feature of a neural net is learning.
`train_network()` function is the function of learning, but how a neural net learns?

#### Backpropagation ####
Backpropagation is the most used technique for training. It changes weights by the weight's error. For example, if a weight changes the prediction too much in a wrong way, Backpropagation will decrease it. If a weight changes the prediction not enough in a right way, Backpropagation will increase it.
 
For more info visit: https://en.wikipedia.org/wiki/Backpropagation

#### Gradient Descent ####
Gradient Descent is a optimization function that we use in deep learning.

For more info watch: https://www.youtube.com/watch?v=IHZwWFHWa-w

NOTE: `train_network()` function can only be used in single-layer networks.

In [8]:
def train_network(X, Y, labels, neural_net, epochs=1000):
    for epoch in range(0, epochs):
        for d in range(0, len(X)):
            prediction = predict_ratio(X[d], neural_net)
            
            # Calculate total error per label
            true_prediction = []
            for i in range(0, len(labels)):
                true_prediction.append(0)
            true_prediction[labels.index(Y[d])] = 1
            
            errors = []
            for t in range(len(prediction)):
                errors.append(true_prediction[t] - prediction[t]) 
            adjust_deriv = errors * sigmoid_deriv(prediction)
            
            for k in range(0, len(adjust_deriv)):
                adjustment = np.dot(X[d], adjust_deriv[k])
                neural_net[0][0][k] += adjustment
    return neural_net

In [9]:
neural_net = train_network(X, Y, labels, neural_net, epochs=1000)

### Results  ###
You may see, neural network can classify the data correctly with a good accuracy rate, but there can be problem named 'Overfitting'. Overfitting means a neural network is fit on the data it trained and cannot classify other possible data points. For better results, train the net with more data and test with data not used in training

In [10]:
for i in range(len(X)):
    print(predict(X[i], neural_net))

(1, 0.9919398375371878)
(1, 0.9936373425420446)
(1, 0.9925307416847557)
(0, 0.9903682943291514)
(0, 0.9836167677309535)
(0, 0.9876082070557368)


You can see the accuracy rate decreses when predicting a data point which is different than training data.

In [11]:
predict([1,1,0,1,0,1,0,1], neural_net)

(0, 0.589227580166354)